In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get list of AI model from "**Console for AITRIOS**"

This notebook explains the workflow for retrieving the list of AI model from "**Console for AITRIOS**". <br>
<br>
Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import pandas as pd
from console_access_library.client import Client
from IPython.display import display

## Load an instance of "**Console Access Library**" Client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [ ]:
%store -r
client_obj: Client = client_obj

## Get AI model list

In [ ]:
# Get an instance of AI model API
ai_model_obj = client_obj.get_ai_model()

# AI model status on Console
status_dictionary = {
    "00": "Conversion project created",
    "01": "Importing completed (Before conversion)",
    "02": "Converting...",
    "03": "Conversion failed",
    "04": "Converted",
    "05": "Adding to configuration",
    "06": "Conversion failed",
    "07": "Conversion completed",
    "11": "Saving",
}

model_ids = []
model_project_names = []
version_numbers = []
version_statuses = []
latest_flags = []

# Call an API for get AI model list
try:
    response = ai_model_obj.get_models()
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
# Create output list
models = response["models"]
for model in models:
    projects = model["projects"]
    for project in projects:
        versions = project["versions"]
        for version in versions:
            model_ids.append(model["model_id"])
            model_project_names.append(project["model_project_name"])
            version_numbers.append(version["version_number"])
            latest_flags.append(version["latest_flg"])
            version_status = version["version_status"]
            version_statuses.append(
                status_dictionary.get(
                    version_status, "Unknown status '" + version_status + "'"
                )
            )
if len(model_ids) == 0:
    raise Exception("AI model list is not found.")

output_frame = pd.DataFrame(
    {
        "model_id": model_ids,
        "project_name": model_project_names,
        "version_number": version_numbers,
        "version_status": version_statuses,
        "latest_flag": latest_flags,
    }
)
# setting backup
backup_max_rows = pd.options.display.max_rows
# output limit clear
pd.set_option("display.max_rows", None)
# View results
display(output_frame)
# setting restore
pd.set_option("display.max_rows", backup_max_rows)

In [ ]:
# Uncomment below to filter by ID
# QUERY_MODEL_ID = "your_model_id"
# output_frame.query(f"model_id == '{QUERY_MODEL_ID}'")